# Assignment 12

### 1.复习上课内容

### 2.回答以下理论问题

#### 1. 请写一下TF-IDF的计算公式

>TF(Term frequency):在一个文档中，一个词出现的次数: $tf_{t,d}=count(t,d)$,也可以对结果取对数, $tf_{t,d}=log_{10}(count(t,d)+1)$<br>
其中，t=某个词在该文档中出现对频次，d=文档的总词数

>IDF(Inverse document frenquency):总文档数除以包含某个词的文档数，再取对数: $idf_t=log_{10}(\frac{N}{df_t})$<br>
其中，N=总文档数，$df_t$=包含t的文档数

>TF-IDF = $TF * IDF$

#### 2. LDA算法的基本假设是什么？

>LDA的2个基本假设：
1. Per-document topic distributions: The topics of each document obey a probability distribution(每个文本主题服从某个概率分布)
2. Per-topic word distributions: The words of each topic obey a probability distribution (每个主题下的词服从一个概率分布)

#### 3. 在TextRank算法中构建图的权重是如何得到的？

>一般是通过计算2个词的词向量之间的余弦相似度来得到的: $W_{ij}=similarity\;cosine(w_i, w_j)$

#### 4. 什么是命名实体识别？ 有什么应用场景？

>- Named entity recognitions(NER): To find each menthon of a named entity in the text and label its type (识别文档中的实体(比如地点，人名，时间等)并加上标签)
- NER types are appliction specific:
    - Commonly include people, places and organizations
    - Biomedical domain: names of genes and proteins
    - Education: codes of college courses
    - ...

#### 5.NLP主要有哪几类任务 ？

>NLP任务:
- 分类任务(Classification tasks)
    - 情感分析
    - 文本分类
    - 实体识别
- 生成任务(Generation tasks)
    - 机器翻译
    - 对话系统(QA)
    - 文本生成

### 3.实践题

#### 3.1 手动实现TextRank算法 (在新闻数据中随机提取100条新闻训练词向量和做做法测试）

 提示：
 1. 确定窗口，建立图链接。   
 2. 通过词向量相似度确定图上边的权重
 3. 根据公式实现算法迭代(d=0.85)

训练词向量

In [7]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import re
import jieba
import random

In [3]:
path = '../lecture5/sqlResult_1558435.csv'
news = pd.read_csv(path, encoding='gb18030')

In [32]:
news = news.fillna('')

In [33]:
content = news['content'].tolist()

In [45]:
# 读取中文停用词
with open('../lecture10/stopwords.dat', 'r') as f:
    stopwords = [w.strip() for w in f.readlines()]

In [52]:
def cut(string):
    return ' '.join(token for token in jieba.cut(''.join(re.sub('\\\\n|[\n\u3000\r]', '', string))) 
                    if token not in stopwords)

In [87]:
# test
cut(content[0])

'本周 月 12 日 小米 手机 15 款 机型 外 机型 暂停 更新 发布 含 开发 版 体验版 内测 稳定版 暂不受 影响 确保 工程师 精力 系统优化 工作 有人 猜测 精力 用到 MIUI   研发 之中 MIUI   去年 月 发布 距今已有 一年 有余 更新换代 MIUI   确切 信息 等待 官方消息'

In [70]:
sample_content = random.sample(content, 100)

In [71]:
new_content = [cut(c) for c in sample_content]

In [72]:
new_content[1]

'新华社 照片 明斯克 2017 年 月 21 日 国际 白俄罗斯 国际 武器装备 军事 技术展 这是 月 20 日 白俄罗斯 首都 明斯克 拍摄 展会 开幕式 跳伞 表演 第八届 白俄罗斯 米 列克斯 2017 国际 武器装备 军事 技术展 20 日 明斯克 开幕 本届 展会 为期 天 吸引 白俄罗斯 俄罗斯 中国 哈萨克斯坦 德国 斯洛伐克 乌克兰 等国 140 多家 参展商 参展 总计 展出 350 多件 武器装备 军用 技术 产品 新华社 记者 魏忠杰 摄'

In [73]:
vectorizer = TfidfVectorizer(token_pattern='(?u)\\b\\w+\\b') # 默认pattern是只匹配长度至少为2的词

In [74]:
X = vectorizer.fit_transform(new_content)

In [75]:
X.shape

(100, 5735)

In [83]:
def get_wv(word):
    return X[:, vectorizer.vocabulary_[word]].toarray()

In [100]:
wordvec_dict = {}
for word in vectorizer.vocabulary_.keys():
    wordvec_dict[word] = get_wv(word)

TextRank算法实现

In [88]:
from collections import defaultdict

In [189]:
class TextRank():
    
    def __init__(self, content, wv):
        self.content = content
        self.wv = wv
        self.wordmap = defaultdict(set)
        
    # 添加关系
    def add_to_wordmap(self, words):
        for i in range(len(words) - 1):
            for j in range(len(words[i:]) - 1):
                # 因为要用词向量计算权重，因此如果某个词没有词向量，则不建立联系
                if words[i] != words[j] and words[i] in self.wv and words[j] in self.wv:
                    self.wordmap[words[i]].add(words[j])
                    self.wordmap[words[j]].add(words[i])
    
    # 根据窗口大小构建关系图
    def createNodes(self, window=3):
        word_list = self.content.split()
        if len(word_list) < window:
            self.add_to_wordmap(word_list)
        else:
            for i in range(len(word_list) - window + 1):
                self.add_to_wordmap(word_list[i:i+window])
    
    def cosine(self, word1, word2):
        wv1 = self.wv[word1]
        wv2 = self.wv[word2]
        return np.dot(wv1.T, wv2)[0][0] / (np.linalg.norm(wv1) * np.linalg.norm(wv2))
    
    # 根据关系构建矩阵
    def createMatrix(self):
        self.words = list(self.wordmap.keys())
        self.matrix = np.zeros([len(self.words), len(self.words)])
        self.word_index = {}
        
        for i, word in enumerate(self.words):
            self.word_index[word] = i
        
        for word_i in self.words:
            for word_j in self.wordmap[word_i]:
                weight = self.cosine(word_i, word_j) / sum([self.cosine(word_j, word_k) for word_k in self.wordmap[word_j]])
                self.matrix[self.word_index[word_i], self.word_index[word_j]] = weight
    
    def calculate(self, num_iters = 1000, d=0.85):
        self.WS = np.ones([len(self.words), 1]) / len(self.words)
        for i in range(num_iters):
            self.WS = (1 - d) + d * np.dot(self.matrix, self.WS)
    
    def get_result(self):
        ws_dict = {}
        for word in self.words:
            ws_dict[word] = self.WS[self.word_index[word]][0]
        return ws_dict

In [357]:
# test
def test_textrank(content, topn=10):
    tk = TextRank(content, wordvec_dict)
    tk.createNodes(window=3)
    tk.createMatrix()
    tk.calculate(num_iters=1000, d=0.85)
    ws = tk.get_result()
    for item in sorted(ws.items(), key=lambda x: x[1], reverse=True)[:10]:
        print('word:{0:{2}<5}, weight:{1:.4f}'.format(item[0], item[1], chr(12288)))

In [361]:
test_textrank(random.choice(new_content))

word:投资　　　, weight:6.2971
word:上市公司　, weight:4.3141
word:资金　　　, weight:4.1200
word:公司　　　, weight:3.9148
word:理财产品　, weight:2.6468
word:闲置　　　, weight:2.1827
word:资产　　　, weight:1.9702
word:理财　　　, weight:1.9613
word:主营业务　, weight:1.9208
word:授权　　　, weight:1.8136


### 3.2 使用词向量和k-means的方法寻找关键词

提示：   
1.使用3.1训练好的词向量   
2.可使用sklearn等机器学习库

In [215]:
from sklearn.cluster import KMeans

In [301]:
class keyword_kmeans():
    
    def __init__(self, train_wv, wv_dict, n_clusters=3, max_iter=500):
        self.centers = KMeans(n_clusters=n_clusters, max_iter=max_iter).fit(train_wv).cluster_centers_
        self.wv_dict = wv_dict
        
    def distance(self, wordvec):
        dist_l = []
        for i, center in enumerate(self.centers):
            dist = np.dot(center, wordvec)[0] / (np.linalg.norm(center) * np.linalg.norm(wordvec))
            dist_l.append((i, dist))
        return max(dist_l, key=lambda x: x[1])
    
    def calculate(self, content, topn=10):
        words = list(set(content.split()))
        self.keywords = sorted([(word, self.distance(self.wv_dict[word])) for word in words if word in self.wv_dict], 
                               key=lambda x:x[1][1], reverse=True)[:topn]
    
    def get_result(self, print_out=True):
        if print_out:
            for word, (center, dist) in self.keywords:
                print('word:{0:{3}<4}, center:{1:<2}, distance:{2:.4f}'.format(word, center, dist, chr(12288)))
        else:
            return self.keywords

In [311]:
kk = keyword_kmeans(X.T, wordvec_dict, n_clusters=3)

In [324]:
kk.calculate(random.choice(new_content))

In [325]:
kk.get_result()

word:新华社　, center:1 , distance:0.9620
word:照片　　, center:1 , distance:0.9600
word:日　　　, center:1 , distance:0.9560
word:月　　　, center:1 , distance:0.9453
word:2017, center:1 , distance:0.9024
word:年　　　, center:1 , distance:0.8927
word:当日　　, center:2 , distance:0.6968
word:记者　　, center:0 , distance:0.5818
word:一场　　, center:0 , distance:0.2282
word:31　　, center:1 , distance:0.2279


这里出现了一些问题，因为所有的新闻里都包含了一些词，比如新华社，日期等，这些都被归类成了一个cluster，进而干扰了后续的聚类结果。尝试去掉这些词后进行训练

In [326]:
stopwords += ['新华社', '日', '月', '2017', '记者', '摄', '照片', '年']

In [327]:
new_content2 = [cut(c) for c in sample_content]

In [330]:
vectorizer2 = TfidfVectorizer(token_pattern='(?u)\\b\\w+\\b') # 默认pattern是只匹配长度至少为2的词
X2 = vectorizer2.fit_transform(new_content2)

In [331]:
X2.shape

(100, 5727)

In [332]:
def get_wv2(word):
    return X2[:, vectorizer2.vocabulary_[word]].toarray()
wordvec_dict2 = {}
for word in vectorizer2.vocabulary_.keys():
    wordvec_dict2[word] = get_wv2(word)

In [339]:
def test_kmeans(content):
    kk = keyword_kmeans(X2.T, wordvec_dict2, n_clusters=6)
    kk.calculate(content)
    kk.get_result()

In [340]:
test_kmeans(random.choice(new_content))

word:轮　　　, center:2 , distance:0.8697
word:甲级联赛, center:2 , distance:0.8508
word:足球　　, center:2 , distance:0.7995
word:客场　　, center:2 , distance:0.7805
word:中　　　, center:5 , distance:0.7800
word:29　　, center:2 , distance:0.7578
word:2016, center:2 , distance:0.7413
word:球员　　, center:2 , distance:0.7134
word:当日　　, center:5 , distance:0.7053
word:进球　　, center:2 , distance:0.7043


对比一下textrank和kmeans

In [365]:
c = random.choice(new_content)
print('TextRank:')
test_textrank(c)
print()
print('Kmeans:')
test_kmeans(c)

TextRank:
word:精神　　　, weight:3.1102
word:飞行　　　, weight:2.8655
word:战略　　　, weight:2.6986
word:新　　　　, weight:2.6001
word:四个　　　, weight:2.3724
word:航天　　　, weight:2.3717
word:圆满成功　, weight:2.3199
word:天舟　　　, weight:1.9669
word:习近平　　, weight:1.9093
word:党　　　　, weight:1.8580

Kmeans:
word:全国　　, center:3 , distance:0.8853
word:27　　, center:3 , distance:0.8068
word:中　　　, center:2 , distance:0.7285
word:届　　　, center:2 , distance:0.6595
word:北京　　, center:3 , distance:0.5266
word:完　　　, center:0 , distance:0.5263
word:日电　　, center:0 , distance:0.5061
word:这是　　, center:3 , distance:0.4838
word:世界　　, center:2 , distance:0.4727
word:中国　　, center:0 , distance:0.4687


简评：
>可以看到，在小样本中，TextRank的效果比kmeans好很多。kmeans的主要问题在于cluster的数量需要人为设定，而且需要手动去掉特殊的停用词。

>kmeans的改进方向：修改distance的计算方法，引入大样本，对不同的cluster进行测试...这里主要就实现一下kmeans提取keywords算法，具体优化就不在这里进行了。

### 3.3  提取新闻人物里的对话。(使用以上提取小数据即可）

提示：    
1.寻找预料里具有表示说的意思。    
2.使用语法分析提取句子结构。    
3.检测谓语是否有表示说的意思。

In [366]:
import pyltp
from gensim.models import Word2Vec
from pyltp import Segmentor
import jieba
from gensim.models.word2vec import LineSentence
from pyltp import  SentenceSplitter,NamedEntityRecognizer,Postagger,Parser,Segmentor
from gensim import models
import numpy as np

In [367]:
cws_model = "ltp_data_v3.4.0/cws.model"
pos_model = "ltp_data_v3.4.0/pos.model"
par_model = "ltp_data_v3.4.0/parser.model"
ner_model = "ltp_data_v3.4.0/ner.model"

In [368]:
def get_word_list(sentence,model):
    #得到分词
    segmentor = Segmentor()
    segmentor.load(model)
    word_list = list(segmentor.segment(sentence))
    segmentor.release()
    return word_list

def get_postag_list(word_list,model):
    #得到词性标注
    postag = Postagger()
    postag.load(model)
    postag_list = list(postag.postag(word_list))
    postag.release()
    return postag_list

def get_parser_list(word_list,postag_list,model):
    #得到依存关系
    parser = Parser()
    parser.load(model)
    arcs = parser.parse(word_list,postag_list)
    arc_list = [(arc.head,arc.relation) for arc in arcs]
    parser.release()
    return arc_list

In [393]:
test_news = random.choice(sample_content)

In [402]:
test_news = re.sub('\\\\n|[\n\u3000\r]', '', test_news)
test_news

'余贺新在颁奖仪式上。新华社记者夏一方摄新华社青岛4月16日新媒体专电（记者张旭东、周欣、张薇）2017年全国游泳冠军赛16日在青岛继续展开较量，里约奥运会银牌得主徐嘉余在打破男子100米仰泳亚洲纪录和200米仰泳全国纪录后，又在半决赛中打破男子50米仰泳全国纪录。伦敦奥运会冠军叶诗文在副项女子200米仰泳决赛中名列第三。男子50米仰泳半决赛中，浙江选手徐嘉余以24秒42名列前茅，打破全国纪录，比此前的全国纪录快了0.16秒。广东队余贺新以22秒46在男子50米自由泳项目中折桂，银牌和铜牌被辽宁队包揽，分属林永庆和老将班豹。余贺新在比赛中出发。新华社记者夏一方摄比赛期间患感冒的余贺新说：“今天状态一般，仅比昨天快了一点点。世锦赛尽力去游，接着就是全运会，赛程挺煎熬的。全运会是零金牌目标，没有包袱。”在女子200米仰泳决赛前，里约奥运会铜牌得主傅园慧在场边大声为自己的队友叶诗文加油。前100米，叶诗文位居头名，第三个50米被浙江队友柳雅欣超越，最后50米湖北队陈洁后来居上。叶诗文在比赛中。新华社记者夏一方摄最终，陈洁夺冠，柳雅欣摘银，叶诗文排名第三。11日打破男子50米蝶泳全国纪录的浙江队李朱濠继续神勇表现，以51秒63摘得100米蝶泳冠军，山东队张麒斌获得第二名，33岁的广东老将周嘉威摘得铜牌。男子100米蝶泳冠军李朱濠（右二），亚军张麒斌（左一）和季军周嘉威（右一）在颁奖仪式上合影新华社记者朱峥摄“现在我更多是享受比赛，作为教练员去比赛，就是欣赏一下李朱濠等比我年轻时水平高的运动员怎么游的，怎么训练的，帮助我教导我的运动员。”周嘉威说。他比决赛第二年龄大的选手大了10岁，比年龄最小的选手大了18岁。15岁的河北小将李冰洁在女子800米自由泳决赛中夺冠，这是她继400米自由泳获胜后的第二金。八一队张雨涵和山东队毕雯馨分获亚军和季军。李冰洁在比赛后。新华社记者夏一方摄赛后，李冰洁的国家队教练刘海涛对弟子的整体表现表示满意。他说：“李冰洁在2015年第一次拿了冠军。她有自己的特性，后来一年的起伏比较大，有一些身体问题需要调节，还有一些技术不足，自己控制不了，需要继续调整，向东京奥运会努力。”上海名将陆滢夺得女子50米蝶泳冠军，江苏队张雨霏和浙江队林欣彤分获第二、第三名。在男女4×100米自由泳接力决赛中，辽宁队夺冠，亚军和季军分属北京队和上海队。编辑：黄杰；签发：高鹏版

In [397]:
def news_parser(news):
    word_list = get_word_list(news,cws_model)
    postag_list = get_postag_list(word_list,pos_model)
    parser_list = get_parser_list(word_list,postag_list,par_model)
    for i in range(len(word_list)):
        print(i+1, word_list[i],parser_list[i])

In [400]:
news_parser(re.sub('\\\\n|[\n\u3000\r]', '', test_news))

1 余贺新 (2, 'SBV')
2 在 (10, 'ADV')
3 颁奖 (4, 'ATT')
4 仪式 (5, 'ATT')
5 上 (2, 'POB')
6 。 (2, 'WP')
7 新华社 (8, 'ATT')
8 记者 (9, 'ATT')
9 夏一方 (10, 'SBV')
10 摄 (0, 'HED')
11 新华社 (17, 'ATT')
12 青岛 (17, 'ATT')
13 4月 (14, 'ATT')
14 16日 (17, 'ATT')
15 新 (16, 'ATT')
16 媒体 (17, 'ATT')
17 专电 (10, 'VOB')
18 （ (20, 'WP')
19 记者 (20, 'ATT')
20 张旭东 (17, 'COO')
21 、 (20, 'WP')
22 周欣 (20, 'COO')
23 、 (22, 'WP')
24 张薇 (20, 'COO')
25 ） (20, 'WP')
26 2017年 (29, 'ATT')
27 全国 (29, 'ATT')
28 游泳 (29, 'ATT')
29 冠军赛 (33, 'SBV')
30 16日 (33, 'ADV')
31 在 (33, 'ADV')
32 青岛 (31, 'POB')
33 继续 (10, 'COO')
34 展开 (33, 'COO')
35 较量 (34, 'VOB')
36 ， (33, 'WP')
37 里约 (38, 'ATT')
38 奥运会 (39, 'ATT')
39 银牌 (40, 'ATT')
40 得主 (41, 'ATT')
41 徐嘉余 (62, 'SBV')
42 在 (62, 'ADV')
43 打破 (56, 'ATT')
44 男子 (49, 'ATT')
45 100 (46, 'ATT')
46 米 (47, 'ATT')
47 仰泳 (49, 'ATT')
48 亚洲 (49, 'ATT')
49 纪录 (55, 'ATT')
50 和 (53, 'LAD')
51 200 (52, 'ATT')
52 米 (53, 'ATT')
53 仰泳 (49, 'COO')
54 全国 (55, 'ATT')
55 纪录 (43, 'VOB')
56 后 (42, 'POB')
57 ， (42, 'WP')


518 ， (516, 'WP')
519 还有 (516, 'COO')
520 一些 (521, 'ATT')
521 技术 (522, 'SBV')
522 不足 (519, 'VOB')
523 ， (519, 'WP')
524 自己 (525, 'SBV')
525 控制 (519, 'COO')
526 不 (527, 'ADV')
527 了 (525, 'CMP')
528 ， (525, 'WP')
529 需要 (525, 'COO')
530 继续 (529, 'VOB')
531 调整 (530, 'VOB')
532 ， (530, 'WP')
533 向 (536, 'ADV')
534 东京 (535, 'ATT')
535 奥运会 (533, 'POB')
536 努力 (530, 'COO')
537 。 (494, 'WP')
538 ” (494, 'WP')
539 上海 (540, 'ATT')
540 名将 (541, 'ATT')
541 陆滢 (542, 'SBV')
542 夺得 (486, 'VOB')
543 女子 (546, 'ATT')
544 50 (545, 'ATT')
545 米 (546, 'ATT')
546 蝶泳 (547, 'ATT')
547 冠军 (542, 'VOB')
548 ， (542, 'WP')
549 江苏队 (550, 'ATT')
550 张雨霏 (555, 'SBV')
551 和 (552, 'LAD')
552 浙江队 (553, 'ATT')
553 林欣彤 (550, 'COO')
554 分 (555, 'ADV')
555 获 (542, 'COO')
556 第二 (559, 'ATT')
557 、 (558, 'WP')
558 第三 (556, 'COO')
559 名 (555, 'VOB')
560 。 (542, 'WP')
561 在 (571, 'ADV')
562 男女 (565, 'ATT')
563 4×100 (564, 'ATT')
564 米 (565, 'ATT')
565 自由泳 (567, 'ATT')
566 接力 (567, 'ATT')
567 决赛 (568, 'ATT')
568 中 (561, 'POB')


问题分解
>1. 找到所有主谓关系: SBV，SBV指向谓语
2. 判断谓语是否是“说”(或者是表示说的词)
3. 如果是，获取该谓语所在的句子以及后面的完整句子

In [403]:
def get_news_parser(news):
    word_list = get_word_list(news,cws_model)
    postag_list = get_postag_list(word_list,pos_model)
    parser_list = get_parser_list(word_list,postag_list,par_model)
    return [(word_list[i], parser_list[i]) for i in range(len(word_list))]

In [405]:
def get_speak(word_parser_list):
    speak_words = ['说'] # 将表示说的词加入这个list
    speak_indexes = []
    for _, (index, parser) in word_parser_list:
        if parser == 'SBV':
            verb_index = index - 1
            if word_parser_list[verb_index][0] in speak_words:
                speak_indexes.append(verb_index)
    return sorted(list(set(speak_indexes)))

In [410]:
def get_speak_sentence(word_parser_list, speak_index):
    """
    这里需要返回2个句子：
        1. “说”这个词所在的句子
            1.1 句首为任意标点符号或者是整个字符串开头
            1.2 句尾为任意标点符号或者是整个字符串结尾
        2. “说”的内容
            2.1 该句应该紧贴着“说“这个词所在的句子（前方或者后方）
            2.2 句首为：，或”
            2.3 句尾为：。或“
            2.4 如果“说”的内容在“说”这个词所在句子之前，那么必须以”结尾
    """
    
    # “说”所在的句子
    speak_start = 0
    for i in range(speak_index, -1, -1):
        if word_parser_list[i][1][1] == 'WP':
            speak_start = i+1
            break
    speak_end = len(word_parser_list) - 1
    for i in range(speak_index, speak_end+1):
        if word_parser_list[i][1][1] == 'WP':
            speak_end = i
            break
    speak_sentence = ''.join(word_parser_list[i][0] for i in range(speak_start, speak_end+1))
    
    # 位于“说”所在句子之前的内容
    content_before = ''
    content_before_end = speak_start - 1
    if content_before_end > 0:
        if word_parser_list[content_before_end][0] == '”': # 没有详细考虑过所有情况，这里假定位于之前的句子内容必须在引号范围内
            for i in range(content_before_end, -1, -1):
                if word_parser_list[i][0] == '“':
                    content_before = ''.join(word_parser_list[i][0] for i in range(i, content_before_end+1))
                    break
    
    # 位于"说"所在句子之后的内容
    content_after = ''
    content_after_start = speak_end + 1
    if word_parser_list[speak_end][0] != '。': # 如果句子以句号结尾，那么说明后面不是说的内容
        if word_parser_list[content_after_start][0] == '“':
            end_wp = '”'
        else:
            end_wp = '。'
        for i in range(content_after_start, len(word_parser_list)):
            if word_parser_list[i][0] == end_wp:
                content_after = ''.join(word_parser_list[i][0] for i in range(content_after_start, i+1))
                break
    
    return speak_sentence, content_before, content_after

In [411]:
def get_speak_content(news):
    word_parser_list = get_news_parser(news)
    speak_indexes = get_speak(word_parser_list)
    if not speak_indexes:
        print('没有人说话！')
    for speak_index in speak_indexes:
        speak_sentence, content_before, content_after = get_speak_sentence(word_parser_list, speak_index)
        print('speak_sentence:')
        print(speak_sentence)
        if content_before:
            print('content_before:')
            print(content_before)
        if content_after:
            print('content_after:')
            print(content_after)
        print()

In [412]:
get_speak_content(test_news)

speak_sentence:
周嘉威说。
content_before:
“现在我更多是享受比赛，作为教练员去比赛，就是欣赏一下李朱濠等比我年轻时水平高的运动员怎么游的，怎么训练的，帮助我教导我的运动员。”

speak_sentence:
他说：
content_after:
“李冰洁在2015年第一次拿了冠军。她有自己的特性，后来一年的起伏比较大，有一些身体问题需要调节，还有一些技术不足，自己控制不了，需要继续调整，向东京奥运会努力。”



>这里出现了一些问题，因为新闻中说的话不止2句，检查后发现是分词将“余贺新说”分成了“余贺”、“新说”而不是“余贺新”、“说”。<br>用jieba分词代替segmentor试试

In [416]:
def get_news_parser2(news):
    word_list = list(jieba.cut(news))
    postag_list = get_postag_list(word_list,pos_model)
    parser_list = get_parser_list(word_list,postag_list,par_model)
    return [(word_list[i], parser_list[i]) for i in range(len(word_list))]

In [417]:
def get_speak_content2(news):
    word_parser_list = get_news_parser2(news)
    speak_indexes = get_speak(word_parser_list)
    if not speak_indexes:
        print('没有人说话！')
    for speak_index in speak_indexes:
        speak_sentence, content_before, content_after = get_speak_sentence(word_parser_list, speak_index)
        print('speak_sentence:')
        print(speak_sentence)
        if content_before:
            print('content_before:')
            print(content_before)
        if content_after:
            print('content_after:')
            print(content_after)
        print()

In [418]:
get_speak_content2(test_news)

speak_sentence:
新华社记者夏一方摄比赛期间患感冒的余贺新说：
content_after:
“今天状态一般，仅比昨天快了一点点。世锦赛尽力去游，接着就是全运会，赛程挺煎熬的。全运会是零金牌目标，没有包袱。”

speak_sentence:
周嘉威说。
content_before:
“现在我更多是享受比赛，作为教练员去比赛，就是欣赏一下李朱濠等比我年轻时水平高的运动员怎么游的，怎么训练的，帮助我教导我的运动员。”

speak_sentence:
他说：
content_after:
“李冰洁在2015年第一次拿了冠军。她有自己的特性，后来一年的起伏比较大，有一些身体问题需要调节，还有一些技术不足，自己控制不了，需要继续调整，向东京奥运会努力。”



>由于新闻语料中包含插入图片的注释，比如第一句中的“新华社记者夏一方摄”。这个需要在清洗数据中进行处理了

>跟换其他新闻试试

In [419]:
get_speak_content2(random.choice(content))

speak_sentence:
勒庞说：
content_after:
“我给马克龙先生打了电话，祝贺他赢得选举。”

speak_sentence:
勒庞说，
content_after:
选举结果对“国民阵线”而言是“历史性的巨大成果”。

speak_sentence:
勒庞说，
content_before:
“我建议，着手深刻政党变革，建立一支全新政治力量，”
content_after:
“我呼吁所有爱国者加入我们。”



In [420]:
get_speak_content2(random.choice(content))

没有人说话！


In [422]:
get_speak_content2(random.choice(content))

speak_sentence:
同事们都说他“
content_after:
能干、认干”，但他却并不满足于此。

speak_sentence:
许政超说。
content_before:
“其实都不是什么大发明，传统器具和作业方式也有不合理的地方，我觉得不顺手就想改一改，说白了其实就是因为自己懒，所以在工作中总寻找省事的办法。”

speak_sentence:
n虽然许政超自己说着“
content_after:
偷懒取巧”，但却搞出了名堂。

speak_sentence:
许政超说。
content_before:
“只有像我一样工作在平凡岗位上的每一位员工都行动起来，才能成就我们的事业，我不愿意做金字塔尖上的人，只想做好一块基石，用自己所掌握的技能和知识，让更多的人在岗位上成长为工匠，这样，我才有存在感和成就感！”



>这里又发现了新的问题，就是句子中包括了"说"这个字，但是并不是真正的说话。

>其他问题肯定还会出现，这个就不像项目一样一一进行处理了